In [1]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

In [2]:
v = CountVectorizer()
v.fit(["Thor Hathodawala is looking for a job"])
v.vocabulary_

{'thor': 5, 'hathodawala': 1, 'is': 2, 'looking': 4, 'for': 0, 'job': 3}

In [6]:
v = CountVectorizer(ngram_range=(2,2)) # bi-gram only
v.fit(["Thor Hathodawala is looking for a job"])
v.vocabulary_

{'thor hathodawala': 4,
 'hathodawala is': 1,
 'is looking': 2,
 'looking for': 3,
 'for job': 0}

In [4]:
v = CountVectorizer(ngram_range=(1,3)) # BOW , and Bi-Gram , Tri-gram
v.fit(["Thor Hathodawala is looking for a job"])
v.vocabulary_

{'thor': 12,
 'hathodawala': 2,
 'is': 5,
 'looking': 9,
 'for': 0,
 'job': 8,
 'thor hathodawala': 13,
 'hathodawala is': 3,
 'is looking': 6,
 'looking for': 10,
 'for job': 1,
 'thor hathodawala is': 14,
 'hathodawala is looking': 4,
 'is looking for': 7,
 'looking for job': 11}

In [7]:
corpus = [
    "Thor ate pizza",
    "Loki is tall",
    "Loki is eating pizza"
]

In [12]:
import spacy
nlp = spacy.load("en_core_web_sm")

def preprocess(text):
    doc = nlp(text)
    
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
            
    return " ".join(filtered_tokens)

preprocess("loki is ate pizza")

'loki eat pizza'

In [14]:
corpus_processed = [preprocess(text) for text in corpus ]
corpus_processed

['thor eat pizza', 'Loki tall', 'Loki eat pizza']

In [16]:
v = CountVectorizer(ngram_range=(1,2))
v.fit(corpus_processed)
v.vocabulary_

{'thor': 7,
 'eat': 0,
 'pizza': 5,
 'thor eat': 8,
 'eat pizza': 1,
 'loki': 2,
 'tall': 6,
 'loki tall': 4,
 'loki eat': 3}

In [19]:
v.transform(["Thor ate pizza"]).toarray()

array([[0, 0, 0, 0, 0, 1, 0, 1, 0]], dtype=int64)

In [66]:
import pandas as pd

In [67]:
df = pd.read_json("Excels\\news_dataset.json")
df.head()

,text,category
0,Watching Schrödinger's Cat Die University of C...,SCIENCE
1,WATCH: Freaky Vortex Opens Up In Flooded Lake,SCIENCE
2,Entrepreneurs Today Don't Need a Big Budget to...,BUSINESS
3,These Roads Could Recharge Your Electric Car A...,BUSINESS
4,Civilian 'Guard' Fires Gun While 'Protecting' ...,CRIME


In [68]:
df['category'].value_counts() # imbalanced

BUSINESS    4254
SPORTS      4167
CRIME       2893
SCIENCE     1381
Name: category, dtype: int64

In [69]:
from imblearn.over_sampling import SMOTE

X = df['text']
y= df['category']
dummies = pd.get_dummies(df['category'],drop_first=True)
df = pd.concat([df,dummies],axis='columns')
df = df.drop("category",axis='columns')

In [70]:
df.head()

,text,CRIME,SCIENCE,SPORTS
0,Watching Schrödinger's Cat Die University of C...,0,1,0
1,WATCH: Freaky Vortex Opens Up In Flooded Lake,0,1,0
2,Entrepreneurs Today Don't Need a Big Budget to...,0,0,0
3,These Roads Could Recharge Your Electric Car A...,0,0,0
4,Civilian 'Guard' Fires Gun While 'Protecting' ...,1,0,0


In [71]:
X = v.fit_transform(X)

In [76]:
X[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [75]:
y = df.drop(['text'],axis='columns')

In [79]:
# df_balanced['category_num'] = df_balanced['category'].map({
#     'BUSINESS': 0,
#     'SPORTS': 1, 
#     'CRIME': 2, 
#     'SCIENCE': 3
# }) # instead of pd.get_dummies()

In [78]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=5,stratify=y)

In [81]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [83]:
X_train.shape

(10156, 194331)

In [84]:
y_train.shape

(10156, 3)

In [91]:
# Replace Count Vectorizer and then apply SMOTE and then train it